In [1]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [2]:
# Custom imports:
from content_collection import collect_serp_data_and_extract_text_from_webpages
from custom_summarize_chain import create_all_summaries, DocumentSummary
from expert_interview_agent import AgentSetup

## Topic Research:

In [5]:
%pip install google-search-results pandas html2text pytest-playwright chromadb nest_asyncio --quiet

Note: you may need to restart the kernel to use updated packages.


In [6]:
!playwright install

In [7]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
# Constant variables:
TOPIC = "Neural networks"
os.environ["SERPAPI_API_KEY"] = ""

In [10]:
# Extract content from webpages into LangChain documents:
text_documents = collect_serp_data_and_extract_text_from_webpages(topic=TOPIC)

In [11]:
# LLM, text splitter + parser:
llm = ChatOpenAI(temperature=0)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=400
)
parser = PydanticOutputParser(pydantic_object=DocumentSummary)

In [12]:
summaries = await create_all_summaries(text_documents, parser, llm, text_splitter)

Summarizing the data!
Summarizing the data!
Summarizing the data!


---

## Expert Interview Questions:


In [13]:
agent = AgentSetup(topic=TOPIC)
result = agent.run(f"My name is James, what is your name?")



> Entering new AgentExecutor chain...
Hello James, I'm OpenAI's language model. I don't have a personal name as I'm an artificial intelligence. How can I assist you today?

> Finished chain.


In [14]:
agent.run(f"What is my name?")



> Entering new AgentExecutor chain...
Your name is James.

> Finished chain.


'Your name is James.'

In [15]:
agent.agent_executor.memory.buffer

[HumanMessage(content='My name is James, what is your name?', additional_kwargs={}, example=False),
 AIMessage(content="Hello James, I'm OpenAI's language model. I don't have a personal name as I'm an artificial intelligence. How can I assist you today?", additional_kwargs={}, example=False),
 HumanMessage(content='What is my name?', additional_kwargs={}, example=False),
 AIMessage(content='Your name is James.', additional_kwargs={}, example=False)]

In [16]:
document_summaries_message = (
        f"document_summaries: {[s.dict() for s in summaries]}"
        f"topic: {TOPIC}"
        f"---"
        f"Use the above to make interview questions, then I want to answer them."
    )
agent.run(document_summaries_message)



> Entering new AgentExecutor chain...

Invoking: `generate_interview_questions` with `{'document_summaries': "[{'concise_summary': 'A neural network can refer to either a neural circuit of biological neurons or a network of artificial neurons. Artificial neural networks are used for solving artificial intelligence problems. They model connections of biological neurons as weights between nodes. These networks can be used for predictive modeling, adaptive control, and self-learning.', 'writing_style': 'The text is informative and provides an overview of neural networks, their applications, and their history.', 'key_points': ['A neural network can refer to a neural circuit of biological neurons or a network of artificial neurons.', 'Artificial neural networks are used for solving artificial intelligence problems.', 'Artificial neural networks model connections of biological neurons as weights between nodes.', 'Positive weights reflect excitatory connections, while negative values mean i

'Great! If you have any more tasks or need further assistance, feel free to ask.'

---

## General Article Outline:


In [17]:
from article_outline_generation import BlogOutlineGenerator
blog_outline_generator = BlogOutlineGenerator(topic=TOPIC)
questions_and_answers = blog_outline_generator.questions_and_answers
outline_result = blog_outline_generator.generate_outline(summaries)

---

## Article Text Generation:


In [18]:
from article_generation import ContentGenerator
content_gen = ContentGenerator(topic=TOPIC, outline=outline_result, questions_and_answers=questions_and_answers)
content_gen.split_and_vectorize_documents(text_documents)
blog_post = content_gen.generate_blog_post()

---

## Title Tag Optimization:


In [ ]:
%pip install thumb --quiet # This is a tool for evaluation of the potential blog post titles.

Note: you may need to restart the kernel to use updated packages.


In [59]:
import os
import thumb

# Set the environment variables (get your API key: https://platform.openai.com/account/api-keys)
os.environ["LANGCHAIN_API_KEY"] = "YOUR_API_KEY_HERE" # optional: for langsmith tracing

In [66]:
base_prompt = f"Generate a title tag for this blog post on topic: {TOPIC}, Title: {outline_result.title}. The first 200 characters: {''.join(blog_post)[0:200]}"

# Set up a prompt templates for the a/b test:
prompt_a = base_prompt + ' ' + "Write the title in a style of {style}. Make sure that it is SEO friendly."
prompt_b = base_prompt + ' ' + "Write the title in a style of {style}. Make sure that is reader friendly. Focus on the reader."

# Set test cases with different input variables:
cases = [
  {"style": "business professional"},
  {"style": "b2b marketing"},
  ]

# Generate the responses
test = thumb.test([prompt_a, prompt_b], cases, runs=10)

HTML(value='"Unlock the Power of Neural Networks: A Comprehensive Guide for Enhanced Pattern Recognition and D…

IntProgress(value=0, description='Progress:', max=40)

Label(value='ThumbTest: 80615803')